In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
!pip install faker

In [4]:
!pip install tqdm

In [5]:
!pip install babel

In [6]:
import os
os.getcwd()


'/home/prateek/conversational_bot'

In [7]:
dataset = []
i = 0
with open("subtitles", 'r') as f:
    for line in f:
        values = line.lower()
        dataset.append(values)
        i = i + 1
        if i == 1000000:
          break
print('Found %s word vectors.' % len(dataset))

Found 1000000 word vectors.


In [8]:
#taken from one of the tensorflow pa
vocab_size = 20000                    
embedding_dim = 200
max_length = 20               
trunc_type='post'
oov_tok = "<OOV>"
MAX_SEQUENCE_LENGTH=20
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(dataset)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(dataset)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type, padding = 'post')

In [9]:
#taken from https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
embeddings_index = {}
with open("glove.6B.200d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
embedding_matrix = np.zeros((len(word_index) + 1,200))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            200,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [11]:
print(dataset[999999])

they made the men dig their own graves.



In [12]:
index_word = {}
for word , index in word_index.items():
    index_word[index] = word

In [13]:
input_seq = []
input_labs = []
j = 0
for i in padded:
  if j%2 == 0:
    input_seq.append(i)
  if j%2 == 1:
    input_labs.append(i)
  j+=1
  if j == 20000:
    break

input_seq = np.array(input_seq)
input_labs = np.array(input_labs)

print(input_seq[8])
print(input_labs[3])
print(input_seq.shape)
print(input_labs.shape)

[ 48  30 268   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
[ 30  87   5 169   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
(10000, 20)
(10000, 20)


In [14]:
print(input_seq[0])

[ 2726   100 12873  1352     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [15]:
print(len(input_labs))

10000


In [16]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [17]:
# Defined shared layers as global variables
repeator = RepeatVector(20)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [18]:
# GRADED FUNCTION: one_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context

In [19]:
n_a = 128 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 128 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.  
# For the purposes of passing the automatic grader
# please do not modify this global variable.  This will be corrected once the automatic grader is also updated.
post_activation_LSTM_cell = LSTM(n_s, return_state = True) # post-attention LSTM 
#output_layer = Dense(len(input_labs), activation=softmax)
output_layer = Dense(20000, activation=softmax)


In [20]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    e = embedding_layer(X)
    
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    #a = Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s) 
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs=context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
        model = Model(inputs=[X,s0,c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [21]:
model = model(20, 20, n_a, n_s, 20000, 20000)


In [22]:
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 200)      9013800     input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 [(None, 128)]        0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 20, 256)      336896      embedding[0][0]                  
___________________________________________________________________________________________

In [23]:
m=10000
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(input_labs.swapaxes(0,1))

In [24]:
opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

In [49]:
model.fit([input_seq, s0, c0], outputs, epochs=5, batch_size=100)

Epoch 1/5
100/100 [==============================] - 54s 542ms/step - loss: 40.6884 - dense_2_loss: 6.2430 - dense_2_1_loss: 6.1259 - dense_2_2_loss: 5.2208 - dense_2_3_loss: 4.4638 - dense_2_4_loss: 3.7071 - dense_2_5_loss: 2.9335 - dense_2_6_loss: 2.3084 - dense_2_7_loss: 1.8941 - dense_2_8_loss: 1.4909 - dense_2_9_loss: 1.1991 - dense_2_10_loss: 0.9610 - dense_2_11_loss: 0.7719 - dense_2_12_loss: 0.6324 - dense_2_13_loss: 0.5375 - dense_2_14_loss: 0.4814 - dense_2_15_loss: 0.4177 - dense_2_16_loss: 0.3697 - dense_2_17_loss: 0.3361 - dense_2_18_loss: 0.3083 - dense_2_19_loss: 0.2860 - dense_2_accuracy: 0.0039 - dense_2_1_accuracy: 0.1492 - dense_2_2_accuracy: 0.3238 - dense_2_3_accuracy: 0.4456 - dense_2_4_accuracy: 0.5556 - dense_2_5_accuracy: 0.6571 - dense_2_6_accuracy: 0.7363 - dense_2_7_accuracy: 0.7949 - dense_2_8_accuracy: 0.8402 - dense_2_9_accuracy: 0.8795 - dense_2_10_accuracy: 0.9089 - dense_2_11_accuracy: 0.9299 - dense_2_12_accuracy: 0.9461 - dense_2_13_accuracy: 0.9571 

In [33]:
#taken from https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
from math import log
from numpy import array
from numpy import argmax

def beam_search_decoder(data, k):
    sequences = [[list(), 0.0]]
    # walk over each step in sequence
    for row in data:
        all_candidates = list()
        row=row.reshape(20000,)
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score - log(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[:k]
    return sequences

In [43]:
h= [("hey there, how are you?")]
seq= tokenizer.texts_to_sequences(h)

In [44]:
print(seq)

[[175, 52, 63, 31, 2]]


In [45]:
pad = pad_sequences(seq,maxlen=20, truncating="post",padding = 'post')

In [46]:
pred = model.predict([pad , np.zeros((1,128)) , np.zeros((1,128))])

In [48]:
ans = beam_search_decoder(pred,3)
ans

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  5.9650143690487445],
 [[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  8.854289221373946],
 [[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  8.97263521536872]]